# CDV displacement and flow field

Lightweight notebook to mirror the CDV tutorial: download trajectories from Hugging Face, build a `TrajectorySet`, extract per-frame displacements, plot a single-frame displacement field, and compute an aggregated flow field with `run_cdv`.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
from trajkit import TrajectorySet
from trajkit.cdv import run_cdv

Download the sample CSV and build a `TrajectorySet`.

In [ ]:
csv_path = hf_hub_download(
    repo_id="m-aban/air-water",
    filename="data/csv/experiment_001_2017-08-16/exp001_t027m_r01um_2017-08-16.csv.gz",
    repo_type="dataset",
)

df = pd.read_csv(csv_path)
ts = TrajectorySet.from_dataframe(
    df,
    dataset_id="cdv_demo",
    track_id_col="id",
    position_cols=["x", "y"],
    time_col="t",
)
len(ts.trajectories)


Compute per-frame displacements at a chosen lag `delta_t` (seconds).

In [ ]:
delta_t = 1.0  # seconds
time_tol = 1e-3

rows = []
for tid, tr in ts.trajectories.items():
    t = tr.time_seconds()
    if len(t) == 0:
        continue
    frames = tr.frame if tr.frame is not None else np.arange(len(t), dtype=int)
    targets = t + delta_t
    idx = np.searchsorted(t, targets, side="left")
    for i, j in enumerate(idx):
        if j >= len(t):
            continue
        if abs(t[j] - targets[i]) > time_tol:
            continue
        dx = tr.x[j] - tr.x[i]
        rows.append(
            {
                "track_id": tid,
                "t": float(t[i]),
                "frame": int(frames[i]),
                "x": float(tr.x[i, 0]),
                "y": float(tr.x[i, 1]),
                "dx": float(dx[0]),
                "dy": float(dx[1]),
            }
        )

disp_df = pd.DataFrame(rows)
print(f"Displacement rows: {len(disp_df)}")


Plot a single-frame displacement displacement field.

In [ ]:
frame0 = disp_df["frame"].min()
df0 = disp_df[disp_df["frame"] == frame0]

plt.figure(figsize=(6, 6))
plt.quiver(
    df0["x"],
    df0["y"],
    df0["dx"],
    df0["dy"],
    angles="xy",
    scale_units="xy",
    scale=1.0,
    width=0.003,
)
plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Displacement field at frame {frame0} (Δt={delta_t}s)")
plt.axis("equal")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


Compute an aggregated flow field with `run_cdv` and plot it.

In [ ]:
# grid for accumulation (rotated source-x frame)
x = np.linspace(-400.0, 400.0, 40)
y = np.linspace(-400.0, 400.0, 40)
X, Y = np.meshgrid(x, y, indexing="xy")
grid = np.stack([X.ravel(), Y.ravel()], axis=1)

mean, sum_w, counts, centers = run_cdv(
    ts,
    delta_t=delta_t,
    grid_centers=grid,
    max_pair_distance=600.0,
    kernel=30.0,
)

U = mean[:, 0].reshape(X.shape)
V = mean[:, 1].reshape(Y.shape)

plt.figure(figsize=(7, 6))
plt.quiver(
    X,
    Y,
    U,
    V,
    angles="xy",
    scale_units="xy",
    scale=1.0,
    width=0.003,
)
plt.xlabel("x (rotated frame)")
plt.ylabel("y (rotated frame)")
plt.title("CDV flow field (aggregated over frames)")
plt.axis("equal")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
